In [ ]:
!pip install remotezip tqdm opencv-python einops
# !pip install -U tensorflow keras

In [ ]:
import cv2
import einops
import numpy as np
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras import layers

In [ ]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [ ]:
# Define the dimensions of one frame in the set of frames created
HEIGHT = 224
WIDTH = 224

In [ ]:
class Conv2Plus1D(keras.layers.Layer):
  def __init__(self, filters, kernel_size, padding):
    """
      A sequence of convolutional layers that first apply the convolution operation over the
      spatial dimensions, and then the temporal dimension.
    """
    super().__init__()
    self.seq = keras.Sequential([
        # Spatial decomposition
        layers.Conv3D(filters=filters,
                      kernel_size=(1, kernel_size[1], kernel_size[2]),
                      padding=padding),
        # Temporal decomposition
        layers.Conv3D(filters=filters,
                      kernel_size=(kernel_size[0], 1, 1),
                      padding=padding)
        ])

  def call(self, x):
    return self.seq(x)

In [ ]:
class ResidualMain(keras.layers.Layer):
  """
    Residual block of the model with convolution, layer normalization, and the
    activation function, ReLU.
  """
  def __init__(self, filters, kernel_size):
    super().__init__()
    self.seq = keras.Sequential([
        Conv2Plus1D(filters=filters,
                    kernel_size=kernel_size,
                    padding='same'),
        layers.LayerNormalization(),
        layers.ReLU(),
        Conv2Plus1D(filters=filters,
                    kernel_size=kernel_size,
                    padding='same'),
        layers.LayerNormalization()
    ])

  def call(self, x):
    return self.seq(x)

In [ ]:
class Project(keras.layers.Layer):
  """
    Project certain dimensions of the tensor as the data is passed through different
    sized filters and downsampled.
  """
  def __init__(self, units):
    super().__init__()
    self.seq = keras.Sequential([
        layers.Dense(units),
        layers.LayerNormalization()
    ])

  def call(self, x):
    return self.seq(x)

In [ ]:
def add_residual_block(input, filters, kernel_size):
  """
    Add residual blocks to the model. If the last dimensions of the input data
    and filter size does not match, project it such that last dimension matches.
  """
  out = ResidualMain(filters,
                     kernel_size)(input)

  res = input
  # Using the Keras functional APIs, project the last dimension of the tensor to
  # match the new filter size
  if out.shape[-1] != input.shape[-1]:
    res = Project(out.shape[-1])(res)

  return layers.add([res, out])

In [ ]:
class ResizeVideo(keras.layers.Layer):
  def __init__(self, height, width):
    super().__init__()
    self.height = height
    self.width = width
    self.resizing_layer = layers.Resizing(self.height, self.width)

  def call(self, video):
    """
      Use the einops library to resize the tensor.

      Args:
        video: Tensor representation of the video, in the form of a set of frames.

      Return:
        A downsampled size of the video according to the new height and width it should be resized to.
    """
    # b stands for batch size, t stands for time, h stands for height,
    # w stands for width, and c stands for the number of channels.
    old_shape = einops.parse_shape(video, 'b t h w c')
    images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
    images = self.resizing_layer(images)
    videos = einops.rearrange(
        images, '(b t) h w c -> b t h w c',
        t = old_shape['t'])
    return videos

In [ ]:
dashing=tf.keras.models.load_model('dashing_model.keras',custom_objects={'Conv2Plus1D':Conv2Plus1D,'ResizeVideo':ResizeVideo,'Project':Project,'ResidualMain':ResidualMain,'add_residual_block':add_residual_block})

In [ ]:
dashing.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 50, 224, 224, 3)]    0         []                            
                                                                                                  
 conv2_plus1d (Conv2Plus1D)  (None, 50, 224, 224, 16)     3152      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 50, 224, 224, 16)     64        ['conv2_plus1d[0][0]']        
 Normalization)                                                                                   
                                                                                                  
 re_lu (ReLU)                (None, 50, 224, 224, 16)     0         ['batch_normalization[0][0

In [ ]:
y=np.array([[1,2],[3,4]])


In [ ]:
y.flatten()
y

array([[1, 2],
       [3, 4]])

In [ ]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install scikit-learn
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio

In [ ]:
app = FastAPI()

In [ ]:
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
@app.get("/")
def read_root():
  return {"Hello": "World"}

In [ ]:
def generator():
  for frames, label in data:
    yield frames, label

    # Define the output signature
output_signature = (tf.TensorSpec(shape=(50, 224, 224, 3), dtype=tf.float32),tf.TensorSpec(shape=(), dtype=tf.int16))

batch_size=8
dataset = tf.data.Dataset.from_generator(generator,output_signature=output_signature).shuffle(buffer_size=100)
test = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
y_pred=dashing.predict(test)
y_pred=y_pred.flatten()

1/1 [==============================] - 27s 27s/step


In [ ]:
from typing import List
import json


# Custom JSON encoder for NumPy arrays
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()  # Convert NumPy arrays to lists
        return super().default(obj)

# Pydantic model for the data structure
class DataItem(BaseModel):
    A: List[List[List[List[float]]]]  # Nested lists representing the 4D array
    B: int  # Integer value

@app.post("/process-data3/")
async def process_data3(indata: List[DataItem]):
  # Convert JSON back to NumPy arrays
  reconstructed_data = [
      (np.array(item.A, dtype=np.float32), np.int64(item.B)) for item in indata
  ]

  def generator():
    for frames, label in reconstructed_data:
      yield frames, label

  # Define the output signature
  output_signature = (tf.TensorSpec(shape=(50, 224, 224, 3), dtype=tf.float32),tf.TensorSpec(shape=(), dtype=tf.int16))

  batch_size=8
  dataset = tf.data.Dataset.from_generator(generator,output_signature=output_signature).shuffle(buffer_size=100)
  test = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
  y_pred=dashing.predict(test)
  y_pred=y_pred.flatten()
  _0s,_1s=0,0
  for a in y_pred:
    if(a<0.5):
      _0s+=1
    else:
      _1s+=1
  ground_truth=False
  if(_1s>=_0s):
    ground_truth=True

  return {
      "message": "Data processed successfully!",
      "_1s":_1s,
      "_0s":_0s,
      "ground_truth":ground_truth
  }


In [ ]:
type(test_ds)

tensorflow.python.data.ops.prefetch_op._PrefetchDataset

In [ ]:
!ngrok config add-authtoken "Your Auth token"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)